In [1]:
import pandas as pd
import os

# Change these to the subjects folder / personal paths
PWD = "/home/dario/repos/malis-project-autism/data/brain_data_processed/output/cpac_singul_pipeline"
ABIDEII_CSV_PATH = "/home/dario/repos/malis-project-autism/data/ABIDEII_Composite_Phenotypic.csv"
FMRI_DIR = "func" # change with "rest_1" as needed

### Reading the correlation matrixes from files ###

subj_dirs = [f"{d}/{FMRI_DIR}" for d in os.listdir(PWD)]
matrixes, sub_ids = list(), list()
for sub in subj_dirs:
    file = [f for f in os.listdir(PWD+"/"+sub) if f.find("CC200_desc-ndmg-1_correlations.csv") > -1]
    if file:
        with open(f"{PWD}/{sub}/{file[0]}", 'r') as f:
            corr_file = f.read().split('\n')[:-1]
        header = corr_file[0].split(',')
        matrix = [[float(x) for x in row.split(',')] for row in corr_file[1:]]
        
        # Excluding 0.0 value matrixes
        if not all(map(lambda x : x == 0.0, matrix[0])):
            matrixes.append(matrix)
            sub_ids.append(int(sub.split('_')[0]))

In [2]:
import numpy as np

### Extracting the features and labels ###

pheno_df = pd.read_csv(ABIDEII_CSV_PATH)
id_labels = dict(zip(pheno_df.SUB_ID, pheno_df.DX_GROUP))
id_sex = dict(zip(pheno_df.SUB_ID, pheno_df.SEX))

X = list()
for matr in matrixes:
    x = list()
    for i in range(len(matr)):
        for j in range(len(matr[i])):
            if j < i:
                x.append(matr[i][j])
    X.append(x)

y = [id_labels[sub] for sub in sub_ids]

sexes = [id_labels[sub] for sub in sub_ids]
# males_X = [x for x, sex in zip(X, sexes) if sex == 1]
# males_y = [label for label, sex in zip(y, sexes) if sex == 1]
# females_X = [x for x, sex in zip(X, sexes) if sex == 2]
# females_y = [label for label, sex in zip(y, sexes) if sex == 2]

In [3]:
### Implementing a dummy classification as baseline ###

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

dummy_classifier_mf = DummyClassifier(strategy="most_frequent")
dummy_classifier_mf.fit(X_train, y_train)

y_hat = dummy_classifier_mf.predict(X_test)
acc_mf = accuracy_score(y_test, y_hat)
print(f"Dummy Accuracy score (most frequent) : {acc_mf}")

dummy_classifier_strat = DummyClassifier(strategy="stratified")
dummy_classifier_strat.fit(X_train, y_train)

y_hat = dummy_classifier_strat.predict(X_test)
acc_strat = accuracy_score(y_test, y_hat)
print(f"Dummy Accuracy score (stratified) : {acc_strat}")

Dummy Accuracy score (most frequent) : 0.4567901234567901
Dummy Accuracy score (stratified) : 0.49382716049382713


In [4]:
### Implementing cross validation ###

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np

k = 5
kf = KFold(n_splits=k, random_state=None)

train_accs_logreg, test_accs_logreg = [], []
train_accs_svc, test_accs_svc = [], []
logreg_models, svc_models = [], []
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(X, y, test_size=0.25)
X_train_out = np.array(X_train_out)
X_test_out = np.array(X_test_out)
y_train_out = np.array(y_train_out)
y_test_out = np.array(y_test_out)

males_X = [x for x, sex in zip(X_test_out, sexes) if sex == 1]
males_y = [label for label, sex in zip(y_test_out, sexes) if sex == 1]
females_X = [x for x, sex in zip(X_test_out, sexes) if sex == 2]
females_y = [label for label, sex in zip(y_test_out, sexes) if sex == 2]
print(f"Outer Test Females: {len(females_X)}\tOuter Test Males: {len(males_X)}")

for train_index , test_index in kf.split(X_train_out):
    # Data splitting
    # X_train , X_test = np.take(X_train_out,train_index,axis=0), np.take(X_test_out,test_index,axis=0)
    # y_train , y_test = np.take(y_train_out,train_index), np.take(y_test_out,test_index)
    X_train , X_test = X_train_out[train_index, :], X_train_out[test_index, :] 
    y_train , y_test = y_train_out[train_index], y_train_out[test_index] 

    # Training on Logistic Regression
    logreg_model = LogisticRegression(solver='liblinear', penalty='l1', C=0.5)
    logreg_model.fit(X_train,y_train)
    pred_values_train = logreg_model.predict(X_train)
    acc_train = accuracy_score(y_train, pred_values_train)
    train_accs_logreg.append(acc_train)

    # Testing on Logistic Regression
    pred_values_test = logreg_model.predict(X_test)
    acc_test = accuracy_score(y_test, pred_values_test)
    test_accs_logreg.append(acc_test)


    # Training on Support Vector Classifier
    svc_model = SVC(kernel="linear", C=0.001)
    svc_model.fit(X_train, y_train)
    acc_train = svc_model.score(X_train, y_train)
    train_accs_svc.append(acc_train)

    # Testing on Support Vector Classifier
    acc_test = svc_model.score(X_test, y_test)
    test_accs_svc.append(acc_test)


    # Saving each fold's model
    logreg_models.append(logreg_model)
    svc_models.append(svc_model)



print("### Logistic Regression results ###")
print(f"Training accuracy of each fold - {train_accs_logreg}")
print(f"Avg train accuracy : {sum(train_accs_logreg)/k}")
print(f"Testing accuracy of each fold - {test_accs_logreg}")
print(f"Avg testing accuracy : {sum(test_accs_logreg)/k}")
print()

print("### SVC results ###")
print(f"Training accuracy of each fold - {train_accs_svc}")
print(f"Avg train accuracy : {sum(train_accs_svc)/k}")
print(f"Testing accuracy of each fold - {test_accs_svc}")
print(f"Avg testing accuracy : {sum(test_accs_svc)/k}")
print()

best_logreg = logreg_models[np.argmax(test_accs_logreg)]
best_svc = svc_models[np.argmax(test_accs_svc)]

print("### Testing on Female subjects ###")
female_score = best_logreg.score(females_X, females_y)
print(f"Testing score on Female subjects (Logistic Regression): {female_score}")
female_score = best_svc.score(females_X, females_y)
print(f"Testing score on Female subjects (SVC): {female_score}")
print()

print("### Testing on Male subjects ###")
male_score = best_logreg.score(males_X, males_y)
print(f"Testing score on Male subjects (Logistic Regression): {male_score}")
male_score = best_svc.score(males_X, males_y)
print(f"Testing score on Male subjects (SVC): {male_score}")
print()


Outer Test Females: 38	Outer Test Males: 43
### Logistic Regression results ###
Training accuracy of each fold - [0.9375, 0.9270833333333334, 0.9375, 0.9322916666666666, 0.921875]
Avg train accuracy : 0.93125
Testing accuracy of each fold - [0.4583333333333333, 0.5833333333333334, 0.4791666666666667, 0.5833333333333334, 0.5833333333333334]
Avg testing accuracy : 0.5375000000000001

### SVC results ###
Training accuracy of each fold - [0.6041666666666666, 0.6041666666666666, 0.59375, 0.65625, 0.6822916666666666]
Avg train accuracy : 0.6281249999999999
Testing accuracy of each fold - [0.4791666666666667, 0.5625, 0.5416666666666666, 0.5833333333333334, 0.5625]
Avg testing accuracy : 0.5458333333333334

### Testing on Female subjects ###
Testing score on Female subjects (Logistic Regression): 0.5
Testing score on Female subjects (SVC): 0.34210526315789475

### Testing on Male subjects ###
Testing score on Male subjects (Logistic Regression): 0.6744186046511628
Testing score on Male subject

In [6]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

model_lr_nested = LogisticRegression(solver="liblinear", penalty="l1")
model_svm_nested = SVC(kernel="linear")

p_grid = {"C": [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 100]}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
clf_lr = GridSearchCV(estimator=model_lr_nested, param_grid=p_grid, cv=cv)
clf_svm = GridSearchCV(estimator=model_svm_nested, param_grid=p_grid, cv=cv)

X, y = np.array(X), np.array(y)
performance_train_lr = []
performance_test_lr = []
performance_train_svm = []
performance_test_svm = []
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X[train_idx, :], X[test_idx, :]
    y_train, y_test = y[train_idx], y[test_idx]

    clf_lr.fit(X_train, y_train)
    preds_train_nested = clf_lr.predict(X_train)
    performance_train_lr.append(accuracy_score(y_train, preds_train_nested))
    preds_test_nested = clf_lr.predict(X_test)
    performance_test_lr.append(accuracy_score(y_test, preds_test_nested))

    clf_svm.fit(X_train, y_train)
    preds_train_nested = clf_svm.predict(X_train)
    performance_train_svm.append(accuracy_score(y_train, preds_train_nested))
    preds_test_nested = clf_svm.predict(X_test)
    performance_test_svm.append(accuracy_score(y_test, preds_test_nested))

observed_acc_train_lr = np.mean(performance_train_lr)
observed_acc_test_lr = np.mean(performance_test_lr)
print("Mean lr training accuracy across folds = %.3f" % observed_acc_train_lr)
print("Mean lr testing accuracy across folds = %.3f" % observed_acc_test_lr)
observed_acc_train_svm = np.mean(performance_train_svm)
observed_acc_test_svm = np.mean(performance_test_svm)
print("Mean svm training accuracy across folds = %.3f" % observed_acc_train_svm)
print("Mean svm testing accuracy across folds = %.3f" % observed_acc_test_svm)